<h1 align = center>SEGMENTING AND CLUSTERING NEIGHBORHOODS IN TORONTO</h1>

In [1]:
# import libraries

import pandas as pd 
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen


import json
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans


!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#  After retreiving the URL and creating a Beautiful soup object

data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

bf = BeautifulSoup(data, 'lxml')

print('Data downloaded!')

Data downloaded!


In [3]:
#Firstly create a list

data = []

#Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.

table = bf.find('table')
table2 = table.find('tbody')

rows = table2.find_all('tr')
for row in rows: #[-1:]:
    cells = row.find_all("td")
    for cell in cells:
        PostalCode = cell.p.b.text
        try:
    
            Borough = cell.i.text
        except: 
            
            try:
                Borough = cell.a.text
            except:
                Borough = cell.span.text
            s = cell.span.text
            
            Neighborhood = s[s.find("(")+1:s.find(")")]
        if Borough != 'Not assigned':
            
# Append to the list
            data.append((PostalCode, Borough, Neighborhood))
            

In [4]:
# **Create a dataframe with list**
df = pd.DataFrame(data)
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

In [6]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# rename the column "PostalCode"
df_data_1.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
df_data_1.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
#Use the csv file to create the final dataframe

df_new = df.merge(df_data_1, on="PostalCode", how="left")
df_new.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h3> PART 3 </h3>
<h4> Explore and cluster the neighborhoods in Toronto.   </h4>


<h4> getting the latitude and longitude values of Toronto </h4>

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


<h4> create a map of Toronto using the latest values  </h4>

In [16]:
new_map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3198cc',
        fill_opacity=0.5).add_to(new_map_toronto)  
    
new_map_toronto

<h4> create a data frame with only Toronto values  </h4>


In [24]:
borough_names = list(df_new.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
# create a new DataFrame with only boroughs that contain the word Toronto
df_new = df_new[df_new['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(df_new.shape)
df_new.head()

(21, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [25]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

 <h4>Foursquare credentials and Version </h4>

In [39]:
CLIENT_ID = 'RVU3VTRAL20BJNWP5Q11INDLB52NMEXIAYSTK0Z33FK4FBTT' # your Foursquare ID
CLIENT_SECRET = 'WEFPNKEPG3CU5J2DKLJIDPJ3KXUBQAZ51J5YVBTGSJNJQ1UO' # your Foursquare Secret
ACCESS_TOKEN = 'LFGIXOBLPAEOYPXLGRAKKRNDEELJZ5PW3Z5N045MB2ROUOGD' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails loaded')
print('CLIENT_ID loaded' )
print('CLIENT_SECRET loaded' )

Your credentails loaded
CLIENT_ID loaded
CLIENT_SECRET loaded


In [44]:
radius = 500
LIMIT = 100
url_foursquare = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
venues = []

for lat, long, post, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['PostalCode'], df_new['Borough'], df_new['Neighborhood']):
    url_foursquare
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

 <h4> insert the list into a new DataFrame </h4>

In [46]:
venues_list_df = pd.DataFrame(venues)

venues_list_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

venues_list_df.head()

,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.65426,-79.360636,DanceLifeX Centre,43.666956,-79.385297,Dance Studio
1,M5A,Downtown Toronto,Regent Park / Harbourfront,43.65426,-79.360636,Fabarnak,43.666377,-79.380964,Restaurant
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.65426,-79.360636,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.65426,-79.360636,Smith,43.666927,-79.381421,Breakfast Spot
4,M5A,Downtown Toronto,Regent Park / Harbourfront,43.65426,-79.360636,Barbara Hall Park,43.666879,-79.381068,Park


In [47]:
venues_list_df.groupby('Neighborhood')['Neighborhood'].count()

Neighborhood
Berczy Park                                                                                                         69
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport    69
Central Bay Street                                                                                                  69
Christie                                                                                                            69
Church and Wellesley                                                                                                69
Commerce Court / Victoria Hotel                                                                                     69
Enclave of M5E                                                                                                      69
First Canadian Place / Underground city                                                                             69
Garden District, Ryerson           

 <h4>how many unique categories are there </h4>

In [49]:
print('There are {} uniques categories.'.format(len(venues_list_df['VenueCategory'].unique())))

There are 48 uniques categories.


Final coment
The exercise identify 48 unique categories 